In [ ]:
!pip install DMBA mord

In [ ]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
import math
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.neural_network import MLPClassifier
import sys
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
print(sys.version)
print('The scikit-learn version is {}.'.format(sklearn.__version__))

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
The scikit-learn version is 1.5.2.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA = Path('/content/drive/My Drive/Colab Notebooks/Analytics Competition/')

In [ ]:
train_X = pd.read_csv(DATA / 'train_X.csv')
valid_X = pd.read_csv(DATA / 'valid_X.csv')
train_y = pd.read_csv(DATA / 'train_y.csv')
valid_y = pd.read_csv(DATA / 'valid_y.csv')

train_X.head()

,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,DisbursementGross,BalanceGross,GrAppv,Franchise_Dummy,RevLineCr_N,...,State_TN,State_VT,State_WA,State_WI,State_WV,IndustryCode_Other,IndustryCode_Real_Estate,UrbanRural_0,UrbanRural_1,UrbanRural_2
0,1996,84,8,0,0,85000.0,0.0,85000.0,0,1,...,0,0,0,0,0,1,0,1,0,0
1,2002,84,7,1,7,50000.0,0.0,50000.0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2005,68,11,0,0,150000.0,0.0,150000.0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2005,69,15,0,15,11600.0,0.0,11600.0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1999,240,14,7,7,221000.0,0.0,221000.0,0,1,...,0,0,0,0,0,0,0,1,0,0


**MLPClassifier**

MlpClassifier is a popular scikit-learn library that uses multi-layer perception algorithm for clasification tasks. These MLPs are an artifical neural network that obersver patterns and make predictions. It is a type of supervised learning which means that it learns from pre labeled data where the output is already known. This is extremly relevant to our loan data set as the classifier can learn patterns from our pre-existing PIF and CHGoff loans and make predictions for futrue data.

Parameter            | Description                                                                             | Example                  | Options                                      
---------------------|-------------------------------------------------------------------------------------|--------------------------|--------------------------------------------
hidden_layer_sizes   | Specifies the number of hidden layers and nodes in each layer.                         | (100, 50)               | -                                           
activation          | Specifies the activation function for the hidden layers.                            | 'relu'                  | 'relu', 'tanh', 'logistic'                
solver              | Specifies the algorithm to use for optimizing the weights of the network.            | 'adam'                  | 'lbfgs', 'sgd', 'adam'                     
max_iter            | Specifies the maximum number of iterations to run the training algorithm for.         | 200                     | -

Solver | Description | Suitable for | Notes
------- | -------- | -------- | --------
'lbfgs' | Quasi-Newton method. | Small datasets. | Can be faster and perform better for small datasets.
'sgd'   | Stochastic Gradient Descent. | Large datasets. | Requires careful tuning of learning rate and momentum.
'adam'  | Adaptive Moment Estimation. | Large datasets. | Generally a good default choice, often performs well without extensive tuning.

Activator | Description | Notes
------- | -------- | --------
'relu'    | Rectified Linear Unit. | A popular choice, often provides good performance.
'tanh'   | Hyperbolic tangent. | Can be useful in certain cases, but 'relu' is often preferred.
'logistic'| Logistic sigmoid. |  Used for binary classification, outputs values between 0 and 1.
'identity'| Identity function (no-op). |  Useful for linear bottleneck, returns f(x) = x.

Solver: 'adam'

Justification: 'adam' is generally a good default choice for large datasets because it often performs well without extensive tuning. It is an adaptive optimization algorithm that automatically adjusts the learning rate for each parameter, which can help to speed up the training process and improve the model's performance. It also combines the benefits of two other popular optimization algorithms, 'sgd' and 'rmsprop'.
Activator: 'relu'

Justification: 'relu' (Rectified Linear Unit) is a widely used activator that often provides good performance in neural networks, especially for large datasets. It is computationally efficient and helps to avoid the vanishing gradient problem, which can hinder the training of deep networks. It also introduces non-linearity into the model, which allows it to learn complex patterns in the data.

[Information Provided by scikit-learn](https://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html)


Pre-Processing

Before running MLP classifier, we must pre-process the data accordingly. You need to scale all you predictors (besides binary dummies) to a 0-1 scale using minmaxscaler.

In [ ]:
numerical_features = ['ApprovalFY', 'Term', 'NoEmp', 'CreateJob', 'RetainedJob', 'DisbursementGross',
                      'BalanceGross', 'GrAppv']

scaler = MinMaxScaler()

scaler.fit(train_X[numerical_features])

train_X[numerical_features] = scaler.transform(train_X[numerical_features])
valid_X[numerical_features] = scaler.transform(valid_X[numerical_features])

We are starting off our model with a single hidden layer for computational efficiency. We will do further comparison to determine the ideal layer quantity.

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(2), activation='relu', solver='adam',
                    max_iter=1000, random_state=1)
#clf.fit(train_X, train_y.values)
clf.fit(train_X, train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(hidden_layer_sizes=2, max_iter=1000, random_state=1)

Our next line searches for the best number of nodes/hidden layers to optimize model preformance on our given data set. Using GridsearchCV applies cross-validation to ensure strong model selection. It will score the various amounts of neurons allowing us to select the most ideal for our model.

This was our ideal code but it was too computationally costly to run on our hardware:

In [ ]:
#param_grid = {
    #'hidden_layer_sizes': [i for i in range(1,20)]
#}
#gridSearch = GridSearchCV(MLPClassifier(activation='relu', solver='adam',
                       # random_state=1, max_iter=10000),
      #  param_grid, cv=10, n_jobs=-1, return_train_score=True)
#gridSearch.fit(train_X, train_y)
#print('Best score: ', gridSearch.best_score_)
#print('Best parameters: ', gridSearch.best_params_)

In [ ]:
#display=['param_hidden_layer_sizes', 'mean_test_score', 'std_test_score']
#print(pd.DataFrame(gridSearch.cv_results_)[display])

In [ ]:
#pd.DataFrame(gridSearch.cv_results_)[display].plot(x='param_hidden_layer_sizes',
                                                   y='mean_test_score', yerr='std_test_score',
                                                   ylim=(0.8, 0.9))
#plt.show()

Obviously we were unable to determine the ideal number of hidden layers. But for the sake of hypthotical, we will set hidden_layer_sizes=(5) and assume this was our best result from the codes and plots above.

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(5), activation='relu', solver='adam', max_iter=500,
                    random_state=1)
clf.fit(train_X, train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(hidden_layer_sizes=5, max_iter=500, random_state=1)

In [ ]:
classificationSummary(train_y, clf.predict(train_X))
classificationSummary(valid_y, clf.predict(valid_X))
train_f1 = f1_score(train_y, clf.predict(train_X))
valid_f1 = f1_score(valid_y, clf.predict(valid_X))

print(f"Training F1 Score: {train_f1}")
print(f"Validation F1 Score: {valid_f1}")

Confusion Matrix (Accuracy 0.8717)

       Prediction
Actual      0      1
     0 425944  18960
     1  50258  44336
Confusion Matrix (Accuracy 0.8718)

       Prediction
Actual      0      1
     0 283914  12788
     1  33312  29652
Training F1 Score: 0.561606181518779
Validation F1 Score: 0.5626351941102804


In [ ]:
best_cutoff_mlp = 0
best_f1_score = 0

for cutoff in np.arange(0.01, 1, 0.01):
    predicted_probs = clf.predict_proba(valid_X)[:, 1]
    predictions = (predicted_probs >= cutoff).astype(int)

    f1 = f1_score(valid_y['Default_CHGOFF'], predictions)

    if f1 > best_f1_score:
        best_f1_score = f1
        best_cutoff_mlp = cutoff

print(f"Best cutoff for MLP Classifier (F1-score): {best_cutoff_mlp}")

Best cutoff for MLP Classifier (F1-score): 0.28


# Conclusions:

Based on the accuracy measures (especially the f1 score), this modeling does not contend for our final model of future loan predictions. However, if it were to applied, the cut off we found is 0.28